# CAI Claude

In [22]:
# %load ./llm.py
import os
import anthropic
import openai
from enum import Enum
from dotenv import load_dotenv
load_dotenv()


anthropic_client = anthropic.Client(os.getenv('ANTHROPIC_API_KEY'))
openai.api_key = os.getenv("OPENAI_API_KEY")


class Model(Enum):
    claude_v1_latest = 'claude-v1'
    claude_v1_0 = "claude-v1.0"
    claude_v1_2 = "claude-v1.2"
    claude_v1_3 = "claude-v1.3"
    claude_instant_v1_latest = "claude-instant-v1"
    claude_instant_v1_0 = "claude-instant-v1.0"
    text_davinci_003 = 'text-davinci-003'
    gpt_3_5_turbo = 'gpt-3.5-turbo'
    gpt_4 = 'gpt-4'


def get_llm_response(prompt: str, model: Model = Model.claude_v1_latest, temp_0_1: float = 0.5):
    # OpenAI uses a temperature range from 0 to 2:
    temp_0_2 = temp_0_1 * 2

    if (model.value == 'text-davinci-003'):
        response = openai.Completion.create(
            model=model.value,
            prompt=f"Human: {prompt}\n\nAssistant: ",
            max_tokens=1024,
            temperature=temp_0_2
        )
        return response['choices'][0]['text'].strip()
    elif (model.value in ['gpt-3.5-turbo', 'gpt-4']):
        response = openai.ChatCompletion.create(
            model=model.value,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=1024,
            temperature=temp_0_2,
            stream=False
        )
        return response['choices'][0]['message']['content']
    else:
        response = anthropic_client.completion(
            prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
            stop_sequences=[anthropic.HUMAN_PROMPT],
            model=model.value,
            max_tokens_to_sample=1024,
            temperature=temp_0_1,
        )
        return response['completion']


In [17]:
model = Model.claude_v1_latest

num_samples = 5

constitution_rules = [
    "Instructions must explicitly describe the tone that the editor adpots.",
    "Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.",
    "Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:\n\n<article>ARTICLE_CONTENT</article>\n\n[Rest of instruction goes here]'.",
    "Instructions which involve multiple steps should be broken down into numbered subtasks.",
    # "Instructions must include explicit examples. For example, if the instruction is to check for spelling errors, it should include an example typo and its correction.",
]
constitution_str = "\n".join(map(lambda x: f"{x[0]+1}. {x[1]}", enumerate(constitution_rules)))


def get_concepts_prompt(n: int):
    return f"Generate a list of {n} tasks that a Copy Editor might do as part of their job, given a written article. The output of each completed task should be written text. Write each task on a separate, numbered line."


def get_naive_response_prompt(concept: str):
    return f"""I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
{concept}
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. Do not say anything else, just respond directly with the instruction for the agent."""


def get_critique_prompt(naive_response: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
{constitution_str}
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write:

The instruction breaks the following rules:
1. Rule: {constitution_rules[0]} - Reason: ..."""


def get_rewrite_prompt(naive_response: str, critique: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
{critique}
</issues>

Rewrite the instruction to address these issues. Respond directly with the rewritten instruction, do not say anything else and do not wrap your answer in <instruction> tags."""


## Generate concepts

In [4]:
concepts_string = get_llm_response(get_concepts_prompt(num_samples), model, 0.7)
print(concepts_string)


 Here is a list of 5 tasks a Copy Editor might do, with the output of each task being edited text:

1. Check for spelling, grammar, and punctuation errors and correct as needed.  

2. Ensure consistent and logical flow and structure to the writing. Rearrange or rewrite sections as needed.  

3. Verify facts and claims, especially if complex or controversial. Research as needed to confirm accuracy and add citations.  

4. Ensure consistent style and tone that matches the publication's standards. Rewrite sections as needed for better flow and clarity.  

5. Read through the entire text for overall coherence and impact. Provide feedback and suggestions for strengthening the writing.


In [16]:
import re


def extract_items(text):
    items = []
    lines = text.split('\n')
    for line in lines:
        stripped = line.strip()
        if stripped and re.match(r'^\d+\.\s', stripped):
            items.append(re.sub(r'^\d+\.\s', '', stripped))
    return items


concepts = extract_items(concepts_string)
if len(concepts) != num_samples:
    raise Exception(f"Expected {num_samples} concepts, but got {len(concepts)}")
else:
    print(f"Got {len(concepts)} concepts")


Got 5 concepts


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [6]:
def get_naive_response(concept):
    return get_llm_response(get_naive_response_prompt(concept), model, 0.3)


# Concurrently (speeds up openai responses, not possible with anthropic)
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     naive_responses = list(executor.map(lambda x: get_naive_response(
#         x[0], x[1]), [(concept, i) for i, concept in enumerate(concepts)]))

# Sequentially
naive_responses = []
for concept in concepts:
    naive_responses.append(get_naive_response(concept))

tasksResponsesString = "\n\n\n===\n\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
print(tasksResponsesString)


TASK:
Check for spelling, grammar, and punctuation errors and correct as needed.

NAIVE RESPONSE:
 Check the provided text for any spelling, grammar, or punctuation errors and make corrections as needed.


===


TASK:
Ensure consistent and logical flow and structure to the writing. Rearrange or rewrite sections as needed.

NAIVE RESPONSE:
 Rearrange and rewrite the text as needed to ensure consistent and logical flow and structure.


===


TASK:
Verify facts and claims, especially if complex or controversial. Research as needed to confirm accuracy and add citations.

NAIVE RESPONSE:
 Verify and cite sources for any facts or claims in the text that are complex or controversial.


===


TASK:
Ensure consistent style and tone that matches the publication's standards. Rewrite sections as needed for better flow and clarity.

NAIVE RESPONSE:
 Rewrite and edit the text to match the publication's style guide and improve flow and clarity.


===


TASK:
Read through the entire text for overall c

Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique

Here's what an example prompt looks like:

In [7]:
print(get_critique_prompt(naive_responses[0]))


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Check the provided text for any spelling, grammar, or punctuation errors and make corrections as needed.
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
1. Instructions must explicitly describe the tone that the editor adpots.
2. Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
3. Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:

<article>ARTICLE_CONTENT</article>

[Rest of instruction goes here]'.
4. Instructions which involve multiple steps should be broken down into numbered subtasks.
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instructi

Now let's execute the critique prompt on each of the naive responses:

In [8]:
def get_critique(naive_response):
    return get_llm_response(get_critique_prompt(naive_response), model, 0)

# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     critiques = list(executor.map(lambda x: get_critique(x[0], x[1]), [
#                      (response, i) for i, response in enumerate(naive_responses)]))


# Sequentially:
critiques = []
for naive_response in naive_responses:
    critiques.append(get_critique(naive_response))

responseCritiqueString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


NAIVE RESPONSE:
 Check the provided text for any spelling, grammar, or punctuation errors and make corrections as needed.

CRITIQUE:
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots. 
- Reason: The instruction does not specify the tone the editor should adopt. It only specifies the tasks of checking for errors and making corrections.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
- Reason: The instruction does not specify a word count for the editor's response. 

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:  

<article>ARTICLE_CONTENT</article>  

[Rest of instruction goes here]'.
- Reason: The instruction does not reference any article content. It only specifies the editing tasks in a general sense.

4. Rule: Instruct

## Rewrite

Here's what an example rewrite prompt looks like:

In [18]:
print(get_rewrite_prompt(naive_responses[0], critiques[0]))


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Check the provided text for any spelling, grammar, or punctuation errors and make corrections as needed.
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots. 
- Reason: The instruction does not specify the tone the editor should adopt. It only specifies the tasks of checking for errors and making corrections.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
- Reason: The instruction does not specify a word count for the editor's response. 

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:  

<article>ARTICLE_CONTENT</article>  

[Rest of 

In [10]:
def get_rewrite(naive_response, critique):
    return get_llm_response(get_rewrite_prompt(naive_response, critique), model, 0.5)

# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     rewrites = list(executor.map(lambda x: get_rewrite(x[0], x[1], x[2]), [
#         (response, critique, i) for i, (response, critique) in enumerate(zip(naive_responses, critiques))]))


# Sequentially:
rewrites = []
for i, (naive_response, critique) in enumerate(zip(naive_responses, critiques)):
    rewrites.append(get_rewrite(naive_response, critique))

rewriteString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nREWRITE:\n{rewrite}" for response, rewrite in zip(naive_responses, rewrites)])
print(rewriteString)


NAIVE RESPONSE:
 Check the provided text for any spelling, grammar, or punctuation errors and make corrections as needed.

REWRITE:
 Here is the rewritten instruction:

1. Here is an article:  

<article> 
The quik brown fox jumpd over the lazy dog. 
</article>

2. Check the provided article for any spelling, grammar, or punctuation errors. Adopt a formal and instructive tone. 

3. Make any necessary corrections to the spelling, grammar, and punctuation in the article. Your response should be around 3 sentences (30-40 words).

4. Ensure any corrections are made clearly and concisely, and the overall meaning and flow of the article is retained.


===


NAIVE RESPONSE:
 Rearrange and rewrite the text as needed to ensure consistent and logical flow and structure.

REWRITE:
 Here is an article:  

<article> 
The history of AI and its progress in the past few decades is truly remarkable. Since the coining of the term "Artificial Intelligence" in 1956, AI has advanced rapidly and become ubiq